# 03. Quick Experiment

This notebook runs a quick test experiment (~5 minutes) to verify the entire pipeline works correctly.

**What it does:**
- Runs 3 rounds of federated learning with 5 clients
- Tests a label flip attack with Krum defense
- Validates accuracy metrics

## Setup

In [ ]:
import sys
import os
import time

# Navigate to project root if in notebooks folder
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

PROJECT_ROOT = os.getcwd()
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
import torch
print(f"Using device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
from experiments.run_experiments import ExperimentConfig, ExperimentRunner

---
## 1. Baseline Experiment (No Attack, No Defense)

In [ ]:
# Create results directory
os.makedirs("./experiments/quick_results", exist_ok=True)

runner = ExperimentRunner("./experiments/quick_results")

In [ ]:
# Baseline config - clean training
baseline_config = ExperimentConfig(
    name="quick_baseline",
    dataset="mnist",
    num_clients=5,
    num_rounds=3,
    local_epochs=1,
    batch_size=32,
    learning_rate=0.01,
    partition="iid",
    seed=42,
    attack_enabled=False,
    defense_enabled=False
)

print("Running baseline experiment...")
start_time = time.time()

baseline_result = runner.run_simulation(baseline_config)

elapsed = time.time() - start_time
print(f"\n{'='*50}")
print(f"BASELINE RESULTS")
print(f"{'='*50}")
print(f"Final Accuracy: {baseline_result.final_accuracy*100:.2f}%")
print(f"Final Loss: {baseline_result.final_loss:.4f}")
print(f"Time: {elapsed:.1f} seconds")

---
## 2. Attack Experiment (Label Flip, No Defense)

In [ ]:
# Attack config - label flip attack
attack_config = ExperimentConfig(
    name="quick_attack",
    dataset="mnist",
    num_clients=5,
    num_rounds=3,
    local_epochs=1,
    batch_size=32,
    learning_rate=0.01,
    partition="iid",
    seed=42,
    # Attack settings
    attack_enabled=True,
    attack_type="label_flip",
    malicious_clients=[0],  # Client 0 is malicious
    poison_ratio=0.3,
    target_class=8,
    defense_enabled=False
)

print("Running attack experiment (label flip)...")
start_time = time.time()

attack_result = runner.run_simulation(attack_config)

elapsed = time.time() - start_time
print(f"\n{'='*50}")
print(f"ATTACK RESULTS")
print(f"{'='*50}")
print(f"Final Accuracy: {attack_result.final_accuracy*100:.2f}%")
print(f"Accuracy Drop: {(baseline_result.final_accuracy - attack_result.final_accuracy)*100:.2f}%")
print(f"Time: {elapsed:.1f} seconds")

---
## 3. Defense Experiment (Label Flip + Krum)

In [ ]:
# Defense config - attack with Krum defense
defense_config = ExperimentConfig(
    name="quick_defense",
    dataset="mnist",
    num_clients=5,
    num_rounds=3,
    local_epochs=1,
    batch_size=32,
    learning_rate=0.01,
    partition="iid",
    seed=42,
    # Attack settings
    attack_enabled=True,
    attack_type="label_flip",
    malicious_clients=[0],
    poison_ratio=0.3,
    target_class=8,
    # Defense settings
    defense_enabled=True,
    defense_type="krum",
    num_malicious_assumed=1
)

print("Running defense experiment (Krum)...")
start_time = time.time()

defense_result = runner.run_simulation(defense_config)

elapsed = time.time() - start_time
print(f"\n{'='*50}")
print(f"DEFENSE RESULTS")
print(f"{'='*50}")
print(f"Final Accuracy: {defense_result.final_accuracy*100:.2f}%")
print(f"Recovery vs Attack: {(defense_result.final_accuracy - attack_result.final_accuracy)*100:.2f}%")
print(f"Time: {elapsed:.1f} seconds")

---
## Summary

In [ ]:
import matplotlib.pyplot as plt

# Compare results
scenarios = ['Baseline\n(Clean)', 'Attack\n(Label Flip)', 'Defense\n(Krum)']
accuracies = [
    baseline_result.final_accuracy * 100,
    attack_result.final_accuracy * 100,
    defense_result.final_accuracy * 100
]

colors = ['#2ecc71', '#e74c3c', '#3498db']

plt.figure(figsize=(8, 5))
bars = plt.bar(scenarios, accuracies, color=colors, edgecolor='black')
plt.ylabel('Accuracy (%)')
plt.title('Quick Experiment Results')
plt.ylim(0, 100)

# Add value labels
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{acc:.1f}%', ha='center', fontsize=12)

plt.tight_layout()
plt.show()

print("\n" + "="*50)
print("QUICK TEST SUMMARY")
print("="*50)
print(f"✓ Baseline accuracy: {accuracies[0]:.2f}%")
print(f"✓ Attack impact: -{(baseline_result.final_accuracy - attack_result.final_accuracy)*100:.2f}%")
print(f"✓ Defense recovery: +{(defense_result.final_accuracy - attack_result.final_accuracy)*100:.2f}%")
print(f"\n✓ Pipeline working correctly!")

---
## Next Steps

The pipeline is working! Proceed to:
- **04_mnist_experiments.ipynb** for full MNIST experiments
- **05_cub200_experiments.ipynb** for CUB-200 experiments